# About the Dataset

The original CORD-19 is a resource of over 1,000,000 scholarly articles, including over 400,000 with full text, about COVID-19, SARS-CoV-2, and related coronaviruses.

In our project, the dataset is sampled from the CORD-19 with size ~10,000 to reduce computation burden.

In [ ]:
import os
import subprocess

# shared link: https://drive.google.com/drive/folders/1Td_ZTUVrsKeftDE5Zll7252YLJdWiNTk?usp=share_link
# you can download the data via the shared link, and skip Step 0 and Step 1 if you want to run the code in your local machine


# Step 0: add the shared folder to your google drive. e.g., /content/drive/MyDrive/CORD_19

# Step 1: Mount Google Drive
from google.colab import drive
drive.mount("/content/drive")


!echo $PWD

!ls /content/drive/MyDrive/CORD_19/

# Step 2: unzip json files
subset_dir = os.path.join(os.getcwd(),  "CORD_19_subset")


zip_file_path="/content/drive/MyDrive/CORD_19/subset.zip"

# Check if the destination directory exists
if not os.path.exists(subset_dir):
    # Unzip the file
    cmd = "unzip {} -d {}".format(zip_file_path, subset_dir)
    proc = subprocess.Popen(cmd, shell=True)
else:
    print(f"Directory {subset_dir} already exists. Skipping extraction.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content
meta_10k.csv  subset  subset.zip
Directory /content/CORD_19_subset already exists. Skipping extraction.


In [ ]:
# import packages


import os
import json
import glob
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt


# Data Load & Pre-processing

In [ ]:
# Load Meta data from meta_10k.csv
data_root = '/content/drive/MyDrive/CORD_19/'

metadata_path = os.path.join(data_root, 'meta_10k.csv')
meta_df = pd.read_csv(metadata_path, dtype={
    'pubmed_id': str,
    'Microsoft Academic Paper ID': str,
    'doi': str
})

print(len(meta_df))
meta_df.head()

9022


,Unnamed: 0,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,44,m71xkuo9,c6bf372c094f035a514975c35a7f9c094abbe493,PMC,Sequence specific visual detection of LAMP rea...,10.1186/1472-6750-6-3,PMC1373654,16401354,cc-by,BACKGROUND: Development of a practical gene po...,2006-01-10,"Mori, Yasuyoshi; Hirano, Tsuyoshi; Notomi, Tsu...",BMC Biotechnol,NaN,NaN,NaN,document_parses/pdf_json/c6bf372c094f035a51497...,document_parses/pmc_json/PMC1373654.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
1,96,hqc7u9w3,c65f0939cf35a0f04bf93bd6e8f771b8521563a5,PMC,Transmission Parameters of the 2001 Foot and M...,10.1371/journal.pone.0000502,PMC1876810,17551582,cc-by,"Despite intensive ongoing research, key aspect...",2007-06-06,"Chis Ster, Irina; Ferguson, Neil M.",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/c65f0939cf35a0f04bf93...,document_parses/pmc_json/PMC1876810.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
2,217,jzwcy7dr,a009c8efa4c5f13a5e604608d4e33e1dac078044,PMC,Results From a Hypothesis Generating Case-Cont...,10.1093/schbul/sbm139,PMC2632504,18156638,bronze-oa,Background: Herpes family viruses can cause ce...,2008-08-20,"Niebuhr, David W.; Millikan, Amy M.; Yolken, R...",Schizophrenia Bulletin,NaN,NaN,NaN,document_parses/pdf_json/a009c8efa4c5f13a5e604...,document_parses/pmc_json/PMC2632504.xml.json,https://academic.oup.com/schizophreniabulletin...,NaN
3,255,02opdk0m,b411e12b20d883ef2ee5ca19d48eff9fccedf05f,PMC,CVTree update: a newly designed phylogenetic s...,10.1093/nar/gkp278,PMC2703908,19398429,cc-by-nc,The CVTree web server (http://tlife.fudan.edu....,2009-07-01,"Xu, Zhao; Hao, Bailin",Nucleic Acids Res,NaN,NaN,NaN,document_parses/pdf_json/b411e12b20d883ef2ee5c...,document_parses/pmc_json/PMC2703908.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,NaN
4,342,094d0rn6,513d5ea4db4eb8e94c14c46b018c6041d78119cf,PMC,IPS-1 Is Essential for the Control of West Nil...,10.1371/journal.ppat.1000757,PMC2816698,20140199,cc-by,The innate immune response is essential for co...,2010-02-05,"Suthar, Mehul S.; Ma, Daphne Y.; Thomas, Sunil...",PLoS Pathog,NaN,NaN,NaN,document_parses/pdf_json/513d5ea4db4eb8e94c14c...,document_parses/pmc_json/PMC2816698.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,NaN


In [ ]:
meta_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9022 entries, 0 to 9021
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        9022 non-null   int64  
 1   cord_uid          9022 non-null   object 
 2   sha               9022 non-null   object 
 3   source_x          9022 non-null   object 
 4   title             9022 non-null   object 
 5   doi               8599 non-null   object 
 6   pmcid             7942 non-null   object 
 7   pubmed_id         6986 non-null   object 
 8   license           9022 non-null   object 
 9   abstract          8289 non-null   object 
 10  publish_time      9022 non-null   object 
 11  authors           9009 non-null   object 
 12  journal           8095 non-null   object 
 13  mag_id            0 non-null      float64
 14  who_covidence_id  0 non-null      float64
 15  arxiv_id          480 non-null    object 
 16  pdf_json_files    9022 non-null   object 


In [ ]:
def glob_files(path, f_type=".json"):
    dst = []
    for root, _, files in os.walk(path):
        for f in files:
            if f.endswith(f_type):
                dst.append(os.path.join(root, f))
    return dst

# glob json files
json_dir = os.path.join(subset_dir, "subset/document_parses/pdf_json/")
print(json_dir)
json_files = glob_files(json_dir, ".json")

print("total json files:", len(json_files))

/content/CORD_19_subset/subset/document_parses/pdf_json/
total json files: 12000


In [ ]:
class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.abstract = []
            self.body_text = []
            # Abstract
            for entry in content['abstract']:
                self.abstract.append(entry['text'])
            # Body text
            for entry in content['body_text']:
                self.body_text.append(entry['text'])
            self.abstract = '\n'.join(self.abstract)
            self.body_text = '\n'.join(self.body_text)

            self.title = content['metadata']['title']

            #dict_keys(['paper_id', 'metadata', 'abstract', 'body_text',
            #'bib_entries', 'ref_entries', 'back_matter'])


    def __repr__(self):
        return f"{self.paper_id}: {self.title } : {self.abstract[:200]}... {self.body_text[:200]}..."


first_row = FileReader(json_files[0])
print(first_row)

3184fc47aab4ea35ab911413018a3a86c56368f1: COVID-19 pneumonia and an indelible ground-glass nodule : ... A 59-year-old man who complained of fever, general malaise, and olfactory and taste disorders was admitted to hospital five days after onset. Coronavirus disease 2019 (COVID-19) pneumonia was diagnose...


In [ ]:
from tqdm import tqdm

def get_breaks(content, length):
    data = ""
    words = content.split(' ')
    total_chars = 0

    # add break every length characters
    for i in range(len(words)):
        total_chars += len(words[i])
        if total_chars > length:
            data = data + "<br>" + words[i]
            total_chars = 0
        else:
            data = data + " " + words[i]
    return data


dict_ = {'paper_id': [], 'doi':[], 'abstract': [], 'body_text': [],
         'authors': [], 'title': [], 'journal': [], 'abstract_summary': []}


for idx, entry in tqdm(enumerate(json_files), total=len(json_files)):
    try:
        content = FileReader(entry)
    except Exception as e:
        continue  # invalid paper format, skip

    # get metadata information
    meta_data = meta_df.loc[meta_df['sha'] == content.paper_id]
    # no metadata, skip this paper
    if len(meta_data) == 0:
        continue
    if len(content.body_text) == 0:
        continue
    dict_['abstract'].append(content.abstract)
    dict_['paper_id'].append(content.paper_id)
    dict_['body_text'].append(content.body_text)
    # also create a column for the summary of abstract to be used in a plot
    if len(content.abstract) == 0:
        # no abstract provided
        dict_['abstract_summary'].append("Not provided.")
    elif len(content.abstract.split(' ')) > 100:
        # abstract provided is too long for plot, take first 300 words append with ...
        info = content.abstract.split(' ')[:100]
        summary = get_breaks(' '.join(info), 40)
        dict_['abstract_summary'].append(summary + "...")
    else:
        # abstract is short enough
        summary = get_breaks(content.abstract, 40)
        dict_['abstract_summary'].append(summary)

    # get metadata information
    meta_data = meta_df.loc[meta_df['sha'] == content.paper_id]

    try:
        # if more than one author
        authors = meta_data['authors'].values[0].split(';')
        if len(authors) > 2:
            # more than 2 authors, may be problem when plotting, so take first 2 append with ...
            dict_['authors'].append(get_breaks('. '.join(authors), 40))
        else:
            # authors will fit in plot
            dict_['authors'].append(". ".join(authors))
    except Exception as e:
        # if only one author - or Null valie
        dict_['authors'].append(meta_data['authors'].values[0])

    # add the title information, add breaks when needed
    try:
        title = get_breaks(meta_data['title'].values[0], 40)
        dict_['title'].append(title)
    # if title was not provided
    except Exception as e:
        dict_['title'].append(meta_data['title'].values[0])

    # add the journal information
    dict_['journal'].append(meta_data['journal'].values[0])

    # add doi
    dict_['doi'].append(meta_data['doi'].values[0])


df_covid = pd.DataFrame(dict_, columns=['paper_id', 'doi', 'abstract', 'body_text',
                                        'authors', 'title', 'journal', 'abstract_summary'])
df_covid.head()

100%|██████████| 12000/12000 [01:05<00:00, 184.15it/s]


,paper_id,doi,abstract,body_text,authors,title,journal,abstract_summary
0,3184fc47aab4ea35ab911413018a3a86c56368f1,10.1002/rcr2.751,,"A 59-year-old man who complained of fever, gen...","Yamanaka, Sumitaka. Ota, Shinichiro. <br>Yos...",COVID‐19 pneumonia and an indelible<br>ground...,Respirol Case Rep,Not provided.
1,97079288bf8a9afb1ef3ee088135df630b2ec2b9,10.1017/s0714980820000343,This article describes priority areas for rese...,"changements, il devint évident que les personn...","Rylett, R. Jane. Alary, Flamine. Goldberg,<...",La COVID-19 et les priorités de recherche sur...,Canadian journal on aging = La revue canadienn...,This article describes priority areas for<br>...
2,4dfc59a66e7a86f6a40c5fafad4c12f483ce40bc,10.3389/fimmu.2012.00378,Pentraxin 3 (PTX3) is a soluble pattern recogn...,The release of neutrophil extracellular traps ...,"Daigo, Kenji. Hamakubo, Takao",Host-protective effect of circulating<br>pent...,Front Immunol,Pentraxin 3 (PTX3) is a soluble pattern<br>re...
3,4bec44342ee1111591970c04aadcff79275981b6,10.4317/jced.58776,Background: With the spread of the COVID-19 vi...,The COVID-19 pandemic imposed challenges for h...,"Garcés-Elías, María-Claudia. León-Manco,<br>...",Impact of mandatory social isolation measures...,J Clin Exp Dent,Background: With the spread of the COVID-19<b...
4,6fa0f00ec5cbe4b9b113408885e5c84898176143,10.3389/fphys.2021.723224,In this article we discuss the concept of acce...,Cholesterol is the dominant sterol in animal c...,"Griffiths, William J.. Wang, Yuqin","Sterols, Oxysterols, and Accessible<br>Choles...",Front Physiol,In this article we discuss the concept of<br>...


In [ ]:
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9022 entries, 0 to 9021
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   paper_id          9022 non-null   object
 1   doi               8599 non-null   object
 2   abstract          9022 non-null   object
 3   body_text         9022 non-null   object
 4   authors           9009 non-null   object
 5   title             9022 non-null   object
 6   journal           8095 non-null   object
 7   abstract_summary  9022 non-null   object
dtypes: object(8)
memory usage: 564.0+ KB


In [ ]:
df = df_covid
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8070 entries, 0 to 9021
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   paper_id          8070 non-null   object
 1   doi               8070 non-null   object
 2   abstract          8070 non-null   object
 3   body_text         8070 non-null   object
 4   authors           8070 non-null   object
 5   title             8070 non-null   object
 6   journal           8070 non-null   object
 7   abstract_summary  8070 non-null   object
dtypes: object(8)
memory usage: 567.4+ KB


In [ ]:
!pip install langdetect

In [ ]:
from tqdm import tqdm
from langdetect import detect
from langdetect import DetectorFactory

# set seed
DetectorFactory.seed = 0

# hold label - language
languages = []

# go through each text
for ii in tqdm(range(0,len(df))):
    # split by space into list, take the first x intex, join with space
    text = df.iloc[ii]['body_text'].split(" ")

    lang = "en"
    try:
        if len(text) > 50:
            lang = detect(" ".join(text[:50]))
        elif len(text) > 0:
            lang = detect(" ".join(text[:len(text)]))
    # ught... beginning of the document was not in a good format
    except Exception as e:
        all_words = set(text)
        try:
            lang = detect(" ".join(all_words))
        # what!! :( let's see if we can find any text in abstract...
        except Exception as e:

            try:
                # let's try to label it through the abstract then
                lang = detect(df.iloc[ii]['abstract_summary'])
            except Exception as e:
                lang = "unknown"
                pass

    # get the language
    languages.append(lang)

100%|██████████| 8070/8070 [01:06<00:00, 121.95it/s]


In [ ]:
from pprint import pprint

languages_dict = {}
for lang in set(languages):
    languages_dict[lang] = languages.count(lang)

print("Total: {}\n".format(len(languages)))
pprint(languages_dict)

Total: 8070

{'de': 20, 'en': 8028, 'es': 13, 'fr': 6, 'id': 1, 'nl': 2}


In [ ]:
df['language'] = languages
df = df[df['language'] == 'en']
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8028 entries, 0 to 9021
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   paper_id          8028 non-null   object
 1   doi               8028 non-null   object
 2   abstract          8028 non-null   object
 3   body_text         8028 non-null   object
 4   authors           8028 non-null   object
 5   title             8028 non-null   object
 6   journal           8028 non-null   object
 7   abstract_summary  8028 non-null   object
 8   language          8028 non-null   object
dtypes: object(9)
memory usage: 627.2+ KB


# Histogram of year / journal

# Map-Reduce

In [ ]:
import csv
import re
from collections import defaultdict
from functools import reduce

# Sample data: list of documents (each document is a string)
documents_MR = []
with open(metadata_path, 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        documents_MR.append(row[4])  # Assuming the 4th column is at index 3 (0-based indexing)
        documents_MR.append(row[9])  # Assuming the 9th column is at index 4 (0-based indexing)


# Specify the words to include in the stoplist
stop_words_upper = ["A", "After", "Also", "An", "And", "As", "At", "Base", "Between", "But", "By", "Did", "Do", "Does", "During", "For", "From", "Have", "Has", "Had", "However",
                    "In", "Is", "It", "Its", "New", "Of", "On", "Or", "That", "The", "To", "Use", "Using", "Was", "We", "Were", "Which", "With", "="]
stop_words_lower = [word.lower() for word in stop_words_upper]

# Mapper function
def map_function(documents_MR):
    # Split the document into words and assign an ID to each word
    for idx, word in enumerate(documents_MR.split(), start=1):
      yield (word.lower(), 1)

# Step 1: Map phase
mapped = []
for documents_MR in documents_MR:
    mapped.extend(map_function(documents_MR))

# Step 2: Shuffle and sort phase (group by key)
shuffled = defaultdict(list)
for word, count in mapped:
    shuffled[word].append(count)

# Step 3: Reduce phase
reduced = {}
for word, counts in shuffled.items():
    reduced[word] = reduce(lambda x, y: x + y, counts)

# Step 4: Merge the counts of words with uppercase and lowercase versions together
merged_counts = defaultdict(int)
for word, count in reduced.items():
    merged_word = re.sub(r'[^a-zA-Z0-9]', '', word)
    # Remove non-alphanumeric characters
    merged_counts[merged_word.lower()] += count

# Step 5.1: filter out the stop words
filtered = {word: count for word, count in reduced.items() if word not in stop_words_upper and word not in stop_words_lower}

# Step 5.2: Sort the filtered words by count in descending order
sorted_words = sorted(filtered.items(), key=lambda x: x[1], reverse=True)
sorted_words_unfilter = sorted(reduced.items(), key=lambda x: x[1], reverse=True)

# Step 5.3: Get the top 50 prevalent words
top50_filtered = sorted_words[:50]
top50_unfilter = sorted_words_unfilter[:80]


# Step 6.1: Print the table of the top 50 prevalent words without stop words list
print("Top 80 Prevalent Words without stopwords:")
print(f"{'ID':<4}{'Word':<20}{'Count'}")
print("-----------------------------")
for idx, (word, count) in enumerate(top50_unfilter, start=1):
    print(f"{idx:<4}{word:<20}{count}")
print("\n"+"\n")

# Step 6.2: Print the table of the top 50 prevalent words with IDs
print("Top 50 Prevalent Words:")
print(f"{'ID':<4}{'Word':<20}{'Count'}")
print("-----------------------------")
for idx, (word, count) in enumerate(top50_filtered, start=1):
    print(f"{idx:<4}{word:<20}{count}")


Top 80 Prevalent Words without stopwords:
ID  Word                Count
-----------------------------
1   the                 97935
2   of                  75152
3   and                 71142
4   in                  46376
5   to                  40334
6   a                   30698
7   with                20595
8   for                 19646
9   is                  12786
10  were                12217
11  that                11841
12  on                  11311
13  was                 11006
14  covid-19            10343
15  this                10102
16  as                  10067
17  we                  9942
18  by                  9331
19  from                8221
20  are                 7789
21  patients            7203
22  an                  7006
23  be                  6202
24  study               5251
25  or                  5157
26  at                  5110
27  have                4582
28  during              4384
29  health              4244
30  has                 4039
31  these   

# Association Analysis

# Similarity Analysis

# Clustering Analysis